In [32]:
# pip install spotipy

In [33]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import numpy as np
import pandas as pd

In [34]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

playlist_index = 0
playlist_uri = 'https://open.spotify.com/playlist/56nEWtxXp6GQh3rpJ1vFNj?si=bbf7147f9a304f9a'
# playlist_uri = 'spotify:playlist:37i9dQZEVXbK4gjvS1FjPY' #top50Singapore
like = True

In [35]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [36]:
# results = sp.user_playlist(username, playlist_id, 'tracks')
results = sp.user_playlist(None, playlist_uri, 'tracks')

In [31]:
playlist_tracks_data = results['tracks']
playlist_tracks_data

{'href': 'https://api.spotify.com/v1/playlists/56nEWtxXp6GQh3rpJ1vFNj/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2020-01-31T06:47:30Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1179789890'},
    'href': 'https://api.spotify.com/v1/users/1179789890',
    'id': '1179789890',
    'type': 'user',
    'uri': 'spotify:user:1179789890'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_group': 'album',
     'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5ndkK3dpZLKtBklKjxNQwT'},
       'href': 'https://api.spotify.com/v1/artists/5ndkK3dpZLKtBklKjxNQwT',
       'id': '5ndkK3dpZLKtBklKjxNQwT',
       'name': 'B.o.B',
       'type': 'artist',
       'uri': 'spotify:artist:5ndkK3dpZLKtBklKjxNQwT'}],
     'available_markets': ['AG',
      'AL',
      'AM',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BE',
      'B

In [22]:
playlist_tracks_data = results['tracks']
playlist_tracks_id = []
playlist_tracks_titles = []
playlist_tracks_artists = []
playlist_tracks_first_artists = []

for track in playlist_tracks_data['items']:
    playlist_tracks_id.append(track['track']['id'])
    playlist_tracks_titles.append(track['track']['name'])
    # adds a list of all artists involved in the song to the list of artists for the playlist
    artist_list = []
    for artist in track['track']['artists']:
        artist_list.append(artist['name'])
    playlist_tracks_artists.append(artist_list)
    playlist_tracks_first_artists.append(artist_list[0])

In [23]:
features = sp.audio_features(playlist_tracks_id)
features_df = pd.DataFrame(data=features, columns=features[0].keys())


In [24]:
features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.701,0.750,1,-4.622,0,0.0456,0.0110,0.000199,0.1250,0.568,113.473,audio_features,5oBzet0pFpm9tEz5pUmdK1,spotify:track:5oBzet0pFpm9tEz5pUmdK1,https://api.spotify.com/v1/tracks/5oBzet0pFpm9...,https://api.spotify.com/v1/audio-analysis/5oBz...,240080,4
1,0.664,0.666,8,-2.203,1,0.0446,0.1930,0.000000,0.0854,0.405,139.987,audio_features,3CYH422oy1cZNoo0GTG1TK,spotify:track:3CYH422oy1cZNoo0GTG1TK,https://api.spotify.com/v1/tracks/3CYH422oy1cZ...,https://api.spotify.com/v1/audio-analysis/3CYH...,210560,4
2,0.694,0.770,6,-5.335,1,0.1490,0.1760,0.000011,0.1180,0.163,125.905,audio_features,0E9ZjEAyAwOXZ7wJC0PD33,spotify:track:0E9ZjEAyAwOXZ7wJC0PD33,https://api.spotify.com/v1/tracks/0E9ZjEAyAwOX...,https://api.spotify.com/v1/audio-analysis/0E9Z...,184560,4
3,0.575,0.758,1,-5.029,0,0.0618,0.0954,0.000000,0.3940,0.493,144.107,audio_features,5JEx7HbmvHQQswJCsoo9rA,spotify:track:5JEx7HbmvHQQswJCsoo9rA,https://api.spotify.com/v1/tracks/5JEx7HbmvHQQ...,https://api.spotify.com/v1/audio-analysis/5JEx...,182623,4
4,0.667,0.891,1,-2.937,0,0.0928,0.1710,0.000603,0.4150,0.456,139.972,audio_features,7zSxnkB1eqbD1u71pH2PY8,spotify:track:7zSxnkB1eqbD1u71pH2PY8,https://api.spotify.com/v1/tracks/7zSxnkB1eqbD...,https://api.spotify.com/v1/audio-analysis/7zSx...,209571,4


In [25]:
features_df['title'] = playlist_tracks_titles
features_df['first_artist'] = playlist_tracks_first_artists
features_df['all_artists'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
features_df = features_df[['id', 'title', 'first_artist', 'all_artists',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]
features_df.head()

,id,title,first_artist,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,5oBzet0pFpm9tEz5pUmdK1,Chandelier (feat. Lauriana Mae),B.o.B,"[B.o.B, Lauriana Mae]",0.701,0.750,1,-4.622,0,0.0110,0.000199,0.1250,0.568,113.473,240080,4
1,3CYH422oy1cZNoo0GTG1TK,Psycho,Red Velvet,[Red Velvet],0.664,0.666,8,-2.203,1,0.1930,0.000000,0.0854,0.405,139.987,210560,4
2,0E9ZjEAyAwOXZ7wJC0PD33,In My Mind,Dynoro,"[Dynoro, Gigi D'Agostino]",0.694,0.770,6,-5.335,1,0.1760,0.000011,0.1180,0.163,125.905,184560,4
3,5JEx7HbmvHQQswJCsoo9rA,Close To Me (with Diplo) (feat. Swae Lee),Ellie Goulding,"[Ellie Goulding, Diplo, Swae Lee]",0.575,0.758,1,-5.029,0,0.0954,0.000000,0.3940,0.493,144.107,182623,4
4,7zSxnkB1eqbD1u71pH2PY8,Drop It Down Like - Radio Edit,Lady Bee,"[Lady Bee, Rachel Kramer]",0.667,0.891,1,-2.937,0,0.1710,0.000603,0.4150,0.456,139.972,209571,4
